In [2]:
import json
import gzip

In [77]:
import csv

In [1]:
# collecting background info about query terms from related matches for annotation

### 1. Generate a file with RM per lemma with query terms

In [12]:
# {lemma: {"query_terms":[], "related_matches":{"wikidata":"","aat":"",pwn:["",""]}}

In [8]:
# import query term
with open('/Users/anesterov/reps/LODlit/query_terms.json','r') as jf:
    query_terms = json.load(jf)

In [4]:
# import RMs
with open('/Users/anesterov/reps/wordsmatter/related_matches/rm.json','r') as jf:
    rm = json.load(jf)

In [22]:
query_terms_related_matches = {}

for lang in query_terms.keys():

    dict_per_lang = {}

    for lemma, terms in query_terms[lang].items():
        
        dict_per_lemma = {}
        dict_per_lemma["query_terms"] = terms
        dict_per_lemma["related_matches"] = {}
        dict_per_lemma["related_matches"]["wikidata"] = []
        dict_per_lemma["related_matches"]["aat"] = []

        for value in rm.values():
            
            # check lang
            if value["lang"] == lang and lemma in value["query_terms"]:
                
                if value["related_matches"]["wikidata"][0] != "None":
                    dict_per_lemma["related_matches"]["wikidata"].append(value["related_matches"]["wikidata"][0])
                if value["related_matches"]["aat"][0] != "None":
                    dict_per_lemma["related_matches"]["aat"].append(value["related_matches"]["aat"][0])
               
                if lang == "en":
                    if value["related_matches"]["pwn"][0] != "None":
                        dict_per_lemma["related_matches"]["pwn"] = value["related_matches"]["pwn"]
                    else:
                        dict_per_lemma["related_matches"]["pwn"] = []
                        
        dict_per_lang[lemma] = dict_per_lemma
        
    query_terms_related_matches[lang] = dict_per_lang

In [ ]:
# connecting related matches from ODWN

In [24]:
with open('/Users/anesterov/reps/LODlit/bg/related_matches_odwn.json','r') as jf:
    odwn_rm = json.load(jf)

In [30]:
for lemma, info in query_terms_related_matches["nl"].items():
    
    info["related_matches"]["odwn"] = {"synset_id":[],"le_id":[]}
    for value in odwn_rm.values():
        if lemma in value["query_terms"]:
            if value["odwn_synsets"] != "":
                info["related_matches"]["odwn"]["synset_id"].extend(value["odwn_synsets"])
            if value["odwn_le"][0] != "None":
                info["related_matches"]["odwn"]["le_id"].extend(value["odwn_le"])

In [32]:
# exporting
with open('lemmas_terms_rms.json', 'w') as jf:
    json.dump(query_terms_related_matches, jf)

### 2. Generate a file with lemmas and literal info of RMs

In [33]:
# importing query results

with gzip.open(f"/Users/anesterov/reps/LODlit/Wikidata/gzip_results_clean_en.json", 'r') as gzip_json:
    wd_en = json.loads(gzip_json.read().decode('utf-8'))
    
with gzip.open(f"/Users/anesterov/reps/LODlit/Wikidata/gzip_results_clean_nl.json", 'r') as gzip_json:
    wd_nl = json.loads(gzip_json.read().decode('utf-8'))
    
with open('/Users/anesterov/reps/LODlit/AAT/aat_query_results_en.json','r') as jf:
    aat_en = json.load(jf)
    
with open('/Users/anesterov/reps/LODlit/AAT/aat_query_results_nl.json','r') as jf:
    aat_nl = json.load(jf)
    
with open('/Users/anesterov/reps/LODlit/PWN/pwn31_query_results.json','r') as jf:
    pwn = json.load(jf)
    
with open('/Users/anesterov/reps/LODlit/ODWN/odwn_query_results.json','r') as jf:
    odwn = json.load(jf)

In [ ]:
# {lemma: {"query_terms":[], "related_matches":{"wikidata":"","aat":"",pwn:["",""]},
#          "related_matches_lit":{"wikidata":[], "aat":[], "pwn":[]}}

In [52]:
# shapind dicts
for lemma, info in query_terms_related_matches["en"].items():
    info["related_matches_lit"] = {}
    info["related_matches_lit"]["wikidata"] = []
    info["related_matches_lit"]["aat"] = []
    info["related_matches_lit"]["pwn"] = []

In [54]:
for lemma, info in query_terms_related_matches["nl"].items():
    info["related_matches_lit"] = {}
    info["related_matches_lit"]["wikidata"] = []
    info["related_matches_lit"]["aat"] = []
    info["related_matches_lit"]["odwn"] = []

In [55]:
# adding lits of wikidata EN

for lemma, info in query_terms_related_matches["en"].items():
    if len(info["related_matches"]["wikidata"]) > 0:
        
        for term in info["query_terms"]:
            if wd_en[term] != []:
                for hit in wd_en[term]:
                    if info["related_matches"]["wikidata"][0] == hit["QID"]:
                        info["related_matches_lit"]["wikidata"].append(hit["prefLabel"])
                        if hit["aliases"] != None:
                            info["related_matches_lit"]["wikidata"].extend(hit["aliases"])
                        if type(hit["description"]) == str:
                            info["related_matches_lit"]["wikidata"].append(hit["description"])
                        if type(hit["description"]) == list:
                            info["related_matches_lit"]["wikidata"].extend(hit["description"])
                        info["related_matches_lit"]["wikidata"].extend(hit["instance_of"])
                        info["related_matches_lit"]["wikidata"].extend(hit["subclass_of"])
                        
                        # if hit with RM is found, don't iterate over other hits
                        break
                        
            # if RM for lemma is found, don't iterate over other query terms
            if len(info["related_matches_lit"]["wikidata"]) > 0:
                break
    continue
    

In [57]:
# adding lits of wikidata NL

for lemma, info in query_terms_related_matches["nl"].items():
    if len(info["related_matches"]["wikidata"]) > 0:
        
        for term in info["query_terms"]:
            if wd_nl[term] != []:
                for hit in wd_nl[term]:
                    if info["related_matches"]["wikidata"][0] == hit["QID"]:
                        info["related_matches_lit"]["wikidata"].append(hit["prefLabel"])
                        if hit["aliases"] != None:
                            info["related_matches_lit"]["wikidata"].extend(hit["aliases"])
                        if type(hit["description"]) == str:
                            info["related_matches_lit"]["wikidata"].append(hit["description"])
                        if type(hit["description"]) == list:
                            info["related_matches_lit"]["wikidata"].extend(hit["description"])
                        info["related_matches_lit"]["wikidata"].extend(hit["instance_of"])
                        info["related_matches_lit"]["wikidata"].extend(hit["subclass_of"])
                        
                        # if hit with RM is found, don't iterate over other hits
                        break
                        
            # if RM for lemma is found, don't iterate over other query terms
            if len(info["related_matches_lit"]["wikidata"]) > 0:
                break
                
    continue

In [59]:
# adding lits of AAT EN

for lemma, info in query_terms_related_matches["en"].items():
    if len(info["related_matches"]["aat"]) > 0:
        
        for term in info["query_terms"]:
            if aat_en[term] != []:
                for hit in aat_en[term]:
                    if info["related_matches"]["aat"][0] == hit["aat_uri"]:
                        info["related_matches_lit"]["aat"].append(hit["prefLabel"])
                        info["related_matches_lit"]["aat"].append(hit["prefLabel_comment"])
                        info["related_matches_lit"]["aat"].extend(hit["altLabel"])
                        info["related_matches_lit"]["aat"].extend(hit["altLabel_comment"])
                        info["related_matches_lit"]["aat"].append(hit["scopeNote"])

                        # if hit with RM is found, don't iterate over other hits
                        break
                        
            # if RM for lemma is found, don't iterate over other query terms
            if len(info["related_matches_lit"]["aat"]) > 0:
                break
    continue

In [61]:
# adding lits of AAT NL

for lemma, info in query_terms_related_matches["nl"].items():
    if len(info["related_matches"]["aat"]) > 0:
        
        for term in info["query_terms"]:
            if aat_nl[term] != []:
                for hit in aat_nl[term]:
                    if info["related_matches"]["aat"][0] == hit["aat_uri"]:
                        info["related_matches_lit"]["aat"].append(hit["prefLabel"])
                        info["related_matches_lit"]["aat"].append(hit["prefLabel_comment"])
                        info["related_matches_lit"]["aat"].extend(hit["altLabel"])
                        info["related_matches_lit"]["aat"].extend(hit["altLabel_comment"])
                        info["related_matches_lit"]["aat"].append(hit["scopeNote"])

                        # if hit with RM is found, don't iterate over other hits
                        break
                        
            # if RM for lemma is found, don't iterate over other query terms
            if len(info["related_matches_lit"]["aat"]) > 0:
                break
    continue

In [63]:
# adding lits of PWN

for lemma, info in query_terms_related_matches["en"].items():
    if len(info["related_matches"]["pwn"]) > 0:
        
        for term in info["query_terms"]:
            if pwn[term] != []:
                for hit in pwn[term]:
                    for hit_id in info["related_matches"]["pwn"]:
                        if hit_id == hit["synset_id"]:
                            info["related_matches_lit"]["pwn"].extend(hit["lemmata"])
                            info["related_matches_lit"]["pwn"].append(hit["definition"])
                            info["related_matches_lit"]["pwn"].extend(hit["examples"])
                        
            # if RM for lemma is found, don't iterate over other query terms
            if len(info["related_matches_lit"]["pwn"]) > 0:
                info["related_matches_lit"]["pwn"] = list(set(info["related_matches_lit"]["pwn"]))
                
                break
    continue

In [70]:
# adding lits of ODWN

for lemma, info in query_terms_related_matches["nl"].items():
    if len(info["related_matches"]["odwn"]["synset_id"]) > 0 or \
    len(info["related_matches"]["odwn"]["le_id"]) > 0:
        
        for term in info["query_terms"]:
            if odwn[term] != []:
                for hit in odwn[term]:

                    if hit["synset_id"] != "" and hit["synset_id"] in info["related_matches"]["odwn"]["synset_id"]:
                        info["related_matches_lit"]["odwn"].append(hit.get("le_written_form"))
                        if hit.get("sense_examples"):
                            info["related_matches_lit"]["odwn"].extend(hit.get("sense_examples"))
                        info["related_matches_lit"]["odwn"].extend(hit["synonyms"])
                        info["related_matches_lit"]["odwn"].extend(hit["synset_definitions"])
                        info["related_matches_lit"]["odwn"].append(hit.get("sense_definition"))
                    if hit["synset_id"] == "" and hit["le_id"] in info["related_matches"]["odwn"]["le_id"]:
                        info["related_matches_lit"]["odwn"].append(hit.get("le_written_form"))
                        if hit.get("sense_examples"):
                            info["related_matches_lit"]["odwn"].extend(hit.get("sense_examples"))
                        info["related_matches_lit"]["odwn"].extend(hit["synonyms"])
                        info["related_matches_lit"]["odwn"].extend(hit["synset_definitions"])
                        info["related_matches_lit"]["odwn"].append(hit.get("sense_definition"))

        info["related_matches_lit"]["odwn"] = list(set(info["related_matches_lit"]["odwn"]))
                
    continue

In [72]:
# exporting file
with open('lemmas_query_terms_related_matches.json', 'w') as jf:
    json.dump(query_terms_related_matches, jf)

### 3. Literals per lemma

In [74]:
with open('/Users/anesterov/reps/LODlit/lemmas_query_terms_related_matches.json','r') as jf:
    rm_lits = json.load(jf)

In [75]:
rm_lits["en"]

{'batavia': {'query_terms': ['batavias', 'batavia'],
  'related_matches': {'wikidata': ['Q1199713'], 'aat': [], 'pwn': []},
  'related_matches_lit': {'wikidata': ['Batavia',
    'capital of the Dutch East Indies (1621...1942), now the city of Jakarta',
    'capital city',
    'toponym'],
   'aat': [],
   'pwn': []}},
 'southern rhodesia': {'query_terms': ['southern rhodesia'],
  'related_matches': {'wikidata': ['Q750583'], 'aat': [], 'pwn': []},
  'related_matches_lit': {'wikidata': ['Southern Rhodesia',
    'self-governing British colony from 1923 to 1980',
    'crown colony',
    'colony'],
   'aat': [],
   'pwn': []}},
 'madras': {'query_terms': ['madras'],
  'related_matches': {'wikidata': ['Q1352'],
   'aat': [],
   'pwn': ['chennai.n.01']},
  'related_matches_lit': {'wikidata': ['Chennai',
    'Madras',
    'megacity and capital of Tamil Nadu, India',
    'city',
    'big city',
    'million city',
    'state capital',
    'business cluster'],
   'aat': [],
   'pwn': ['Madras',
 

In [78]:
with open('related_matches_literals_en.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    header = ["lemma","source_1","source_2","source_3"]
    writer.writerow(header)
    
    for lemma, info in rm_lits["en"].items():
        data = [lemma,info["related_matches_lit"]["wikidata"],\
                info["related_matches_lit"]["aat"],\
                info["related_matches_lit"]["pwn"]]
    
        writer.writerow(data)

In [79]:
with open('related_matches_literals_nl.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    header = ["lemma","source_1","source_2","source_3"]
    writer.writerow(header)
    
    for lemma, info in rm_lits["nl"].items():
        data = [lemma,info["related_matches_lit"]["wikidata"],\
                info["related_matches_lit"]["aat"],\
                info["related_matches_lit"]["odwn"]]
    
        writer.writerow(data)